# 0A_download_nci_data

makes a `final_all_data.json` then processes the diseases, biomarkers, etc into `interventional_trials_simple_dx_bm...`

with datetime

In [11]:
import requests
import json
import pandas as pd
from datetime import datetime
from collections import defaultdict, Counter
import pickle


In [12]:
def simplify_search_results(data, concept_type) -> dict:
    '''
    Simplify the output to only code, name, type, and eligibility criteria
    Also update the concept type from "include_X" -> "X" or "exclude_X" -> "X_not"
    '''
    thesaurus = st.session_state.thesaurus
    keep_types = {"disease":['Neoplastic Process'],
                "biomarker":['Cell or Molecular Dysfunction','Finding'],
                "intervention":['Therapeutic or Preventive Procedure','Chemical Viewed Functionally']}

    relevant_types = [keep_types[k] for k in keep_types if k in concept_type][0]
    thesaurus = thesaurus[thesaurus['type'].isin(relevant_types)]
    valid_codes = set(thesaurus['Serial'])

    simplified_data = []
    for search_result in data:
        simplified_concepts = [
            {
                'code': concept['code'],
                'name': concept['name']
            }
            for concept in search_result['concepts']
            if concept['code'] in valid_codes
        ]
        simplified_data.extend(simplified_concepts)

    clean_concept_type = concept_type.split("exclude_")[-1]+".nci_thesaurus_concept_id"+"_not" if "exclude_" in concept_type else concept_type.split("include_")[-1]+".nci_thesaurus_concept_id"
    #clean_concept_type = concept_type.split("_")[-1]+".nci_thesaurus_concept_id"

    return {clean_concept_type:[s["code"] for s in simplified_data]}

In [13]:
import requests
import json
import time
from datetime import datetime

url = "https://clinicaltrialsapi.cancer.gov/api/v2/trials"
headers = {
    "Content-Type": "application/json",
    "x-api-key": "NibhGQRcr84CrgDHkego51aqrNioOR1F7k6f7dNp"
}

default_params = {
    "from": 0,
    "size": 50,
    "record_verification_date_gte": '2014-08-25'
}

response = requests.get(url, params=default_params, headers=headers)

curr_data = response.json()
#all_data = curr_data["data"]
total = curr_data["total"]

KeyError: 'total'

In [14]:
curr_data

{'message': 'Internal server error'}

In [ ]:

# Define a function to save data to a file
def save_data_to_file(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)
def process_prior_therapy(therapies):
    structured_dict = {
        'inclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        },
        'exclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        }
    }

    for therapy in therapies:
        if therapy['nci_thesaurus_concept_id'] is not None:
            # Default to 'inclusion' and 'TREE' if keys are missing or values are None
            eligibility = therapy.get('eligibility_criterion', 'inclusion')
            indicator = therapy.get('inclusion_indicator', 'TREE')

            # Additional check to ensure eligibility and indicator have valid values
            if eligibility not in structured_dict:
                eligibility = 'inclusion'  # Default to 'inclusion' if the value is not recognized
            if indicator not in structured_dict[eligibility]:
                indicator = 'TREE'  # Default to 'TREE' if the value is not recognized

            concept_id = therapy['nci_thesaurus_concept_id']
            ancestors = therapy.get('ancestors', [])
            synonyms = therapy.get('synonyms', [])
            name = therapy.get('name', '')
            entry = structured_dict[eligibility][indicator]
            entry['nci_thesaurus_concept_id'].append(concept_id)
            entry['ancestors'].extend(ancestors)  # Assuming you want to flatten the ancestors list
            entry['synonyms'].extend(synonyms)  # Same for synonyms
            entry['names'].append(name)
     # Convert sets back to lists before returning
    for eligibility_status in structured_dict:
        for indicator_type in structured_dict[eligibility_status]:
            for key in structured_dict[eligibility_status][indicator_type]:
                structured_dict[eligibility_status][indicator_type][key] = list(set(structured_dict[eligibility_status][indicator_type][key]))

    return structured_dict



def process_interventions(arms):
    structured_dict = {
        'inclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        },
        'exclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        }
    }
    arms
    for arm in arms:
        for i in range(len(arm)):
            if 'interventions' in arm.keys():
                interventions = arm['interventions']
                for intervention in interventions:
                    if intervention['nci_thesaurus_concept_id'] is not None:
                        # Default to 'inclusion' and 'TREE' if keys are missing or values are None
                        eligibility = intervention.get('eligibility_criterion', 'inclusion')
                        indicator = intervention.get('inclusion_indicator', 'TREE')

                        # Additional check to ensure eligibility and indicator have valid values
                        if eligibility not in structured_dict:
                            eligibility = 'inclusion'  # Default to 'inclusion' if the value is not recognized
                        if indicator not in structured_dict[eligibility]:
                            indicator = 'TREE'  # Default to 'TREE' if the value is not recognized

                        concept_id = intervention['nci_thesaurus_concept_id']
                        ancestors = intervention.get('ancestors', [])
                        parents = intervention.get('parents', [])
                        synonyms = intervention.get('synonyms', [])
                        name = intervention.get('name', '')
                        entry = structured_dict[eligibility][indicator]
                        entry['nci_thesaurus_concept_id'].append(concept_id)
                        entry['ancestors'].extend(ancestors)  # Assuming you want to flatten the ancestors list
                        entry['synonyms'].extend(synonyms)  # Same for synonyms
                        entry['names'].append(name)
                        entry['parents'].extend(parents)
    # Convert sets back to lists before returning
    for eligibility_status in structured_dict:
        for indicator_type in structured_dict[eligibility_status]:
            for key in structured_dict[eligibility_status][indicator_type]:
                #print(structured_dict[eligibility_status][indicator_type][key])
                structured_dict[eligibility_status][indicator_type][key] = list(set(structured_dict[eligibility_status][indicator_type][key]))

    return structured_dict

def process_biomarkers(biomarkers):
    structured_dict = {
        'inclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        },
        'exclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        }
    }

    for biomarker in biomarkers:
        if biomarker['nci_thesaurus_concept_id'] is not None:
            # Default to 'inclusion' and 'TREE' if keys are missing or values are None
            eligibility = biomarker.get('eligibility_criterion', 'inclusion')
            indicator = biomarker.get('inclusion_indicator', 'TREE')

            # Additional check to ensure eligibility and indicator have valid values
            if eligibility not in structured_dict:
                eligibility = 'inclusion'  # Default to 'inclusion' if the value is not recognized
            if indicator not in structured_dict[eligibility]:
                indicator = 'TREE'  # Default to 'TREE' if the value is not recognized

            concept_id = biomarker['nci_thesaurus_concept_id']
            ancestors = biomarker.get('ancestors', [])
            parents = biomarker.get('parents', [])
            synonyms = biomarker.get('synonyms', [])
            name = biomarker.get('name', '')
            entry = structured_dict[eligibility][indicator]
            entry['nci_thesaurus_concept_id'].append(concept_id)
            entry['ancestors'].extend(ancestors)  # Assuming you want to flatten the ancestors list
            entry['synonyms'].extend(synonyms)  # Same for synonyms
            entry['names'].append(name)
            entry['parents'].extend(parents)
    # Convert sets back to lists before returning
    for eligibility_status in structured_dict:
        for indicator_type in structured_dict[eligibility_status]:
            for key in structured_dict[eligibility_status][indicator_type]:
                structured_dict[eligibility_status][indicator_type][key] = list(set(structured_dict[eligibility_status][indicator_type][key]))

    return structured_dict

def process_diseases(diseases):
    structured_dict = {
        'inclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        },
        'exclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        }
    }

    for dx in diseases:
        # Default to 'inclusion' and 'TREE' if keys are missing or values are None
        eligibility = dx.get('is_lead_disease', False)
        indicator = dx.get('inclusion_indicator', 'TREE')

        # Additional check to ensure eligibility and indicator have valid values
        if eligibility not in structured_dict:
            eligibility = 'inclusion'  # Default to 'inclusion' if the value is not recognized
        if indicator not in structured_dict[eligibility]:
            indicator = 'TREE'  # Default to 'TREE' if the value is not recognized
        concept_id = dx['nci_thesaurus_concept_id']
        ancestors = dx.get('ancestors', [])
        parents = dx.get('parents', [])
        synonyms = dx.get('synonyms', [])
        name = dx.get('name', '')
        
        if name is not None:
            entry = structured_dict[eligibility][indicator]
            entry['synonyms'].extend(synonyms)  # Same for synonyms
            entry['nci_thesaurus_concept_id'].append(concept_id)
            entry['ancestors'].extend(ancestors)  # Assuming you want to flatten the ancestors list

            entry['names'].append(name)
            entry['parents'].extend(parents)
    # Convert sets back to lists before returning
    for eligibility_status in structured_dict:
        for indicator_type in structured_dict[eligibility_status]:
            for key in structured_dict[eligibility_status][indicator_type]:
                structured_dict[eligibility_status][indicator_type][key] = list(set(structured_dict[eligibility_status][indicator_type][key]))

    return structured_dict

In [6]:
%%time

all_data = []
# Loop to fetch data
for i in range(50, total, 50):
    default_params["from"] = i
    time.sleep(1)  # Rate limiting precaution
    response = requests.get(url, params=default_params, headers=headers)
    all_data.extend(response.json()["data"])
    print(i)
    
    # Save every 10 loops
    if (i % 1000) == 0:  # Using `i // 50` to get the loop count starting from 1
        # Generate a filename with a timestamp
        filename = f'all_data_backup_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.json'
        save_data_to_file({"total": total, "data": all_data}, filename)
        print(f'Data backup saved to {filename}')

# Final save for any remaining data
final_filename = 'final_all_data.json'
save_data_to_file({"total": total, "data": all_data}, final_filename)
print(f'Final data saved to {final_filename}')

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
Data backup saved to all_data_backup_2024-12-26_11-28-16.json
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
Data backup saved to all_data_backup_2024-12-26_11-29-55.json
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
Data backup saved to all_data_backup_2024-12-26_11-31-30.json
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
Data backup saved to all_data_backup_2024-12-26_11-33-38.json
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
Data backup saved to all_data_backup_2024-12-26_11-36-16.json
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
Data backup saved to all_data_backup_2024-12-26_11-38-16.json
6050
6100
6150
6200
6250
6300
6350
6400
6450
650

In [7]:
final_filename

'final_all_data.json'

In [8]:
import time

In [9]:
test_list = list()
for i in range(len(all_data)):
    test = all_data[i]['why_study_stopped']
    if test is not None:
        test_list.append(test)

In [40]:
len(all_data)

23663

In [11]:
active_trials = [all_data[i] for i in range(len(all_data)) if all_data[i]['current_trial_status']=='Active']
len(active_trials)

6464

In [12]:
name_file = f'active_trials_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.json'

save_data_to_file({"total": total, "data": active_trials}, name_file)

In [13]:
interventional_trials = [all_data[i] for i in range(len(all_data)) if all_data[i]['study_protocol_type']=='Interventional']

print(len(interventional_trials))

active_interventional_trials = [interventional_trials[i] for i in range(len(interventional_trials)) if interventional_trials[i]['current_trial_status']=='Active']
print(len(active_interventional_trials))

21554
5588


In [14]:
%%time
name_file = f'interventional_trials_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.json'
save_data_to_file({"total": len(interventional_trials), "data": interventional_trials}, name_file)
name_file = f'active_interventional_trials_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.json'
save_data_to_file({"total": len(active_interventional_trials), "data": active_interventional_trials}, name_file)


CPU times: user 1min 13s, sys: 2.09 s, total: 1min 15s
Wall time: 1min 15s


In [15]:
import glob
print (glob.glob("active*.json"))
print (glob.glob("final*.json"))
print (glob.glob("*interventional*.json"))

['active_trials_2024-12-26_12-10-14.json', 'active_interventional_trials_2024-12-26_12-11-31.json', 'active_interventional_trials_2024-07-26_13-02-57.json', 'active_trials_2024-07-24_20-48-27.json', 'active_trials_simple_dx_bm_2024-07-24_23-09-52.json']
['final_all_data.json']
['interventional_trials_2024-12-26_12-10-35.json', 'interventional_trials_with_descendants2024-07-26.json', 'active_interventional_trials_2024-12-26_12-11-31.json', 'interventional_trials_with_descendants2024-08-23.json', 'interventional_trials_2024-07-26_13-02-20.json', 'active_interventional_trials_2024-07-26_13-02-57.json', 'filtered_interventional_trials.json', 'interventional_trials_simple_dx_bm_2024-07-26_13-14-44.json']


# below is making dx_bm for stuff

In [17]:
import json
# trials = json.load(open('interventional_trials_2024-07-26_13-02-20.json','r'))
trials = json.load(open('interventional_trials_2024-12-26_12-10-35.json','r'))




In [92]:
# import json
# import os

# # Load the JSON data from a file
# # with open('active_trials_2024-07-24_20-48-27.json', 'r') as file:
# with open('interventional_trials_2024-07-26_13-02-20.json', 'r') as file:
#     json_data = json.load(file)

# # Check if "data" key exists
# if 'data' in json_data:
#     data = json_data['data']
    
#     # Define the latitude and longitude range
#     min_latitude = 36.8
#     max_latitude = 38.6
#     min_longitude = -123.0
#     max_longitude = -121.5

#     # Check if a site is within the specified range
#     def is_within_range(site, min_lat, max_lat, min_lon, max_lon):
#         if site and 'org_coordinates' in site:
#             if site['org_coordinates'] is not None:
#                 if 'lat' in site['org_coordinates'] and 'lon' in site['org_coordinates']:
#                     lat = site['org_coordinates']['lat']
#                     lon = site['org_coordinates']['lon']
#                     return min_lat <= lat <= max_lat and min_lon <= lon <= max_lon
#                 else:
#                     return False
#             else:
#                 return False
#         else:
#             return False

#     # Filter items in data where at least one site is within the specified range
#     filtered_items = []
#     for item in data:
#         if 'sites' in item and isinstance(item['sites'], list):
#             sites = item['sites']
#             if any(is_within_range(site, min_latitude, max_latitude, min_longitude, max_longitude) for site in sites):
#                 filtered_items.append(item)
    
#     # New dictionary with the filtered items
#     filtered_data = {'data': filtered_items}

#     output_file_path = 'filtered_interventional_trials.json'
#     with open(output_file_path, 'w') as file:
#         json.dump(filtered_data, file, indent=4)

#     # Print the absolute path of the output file
#     print("Filtered data has been saved to:", os.path.abspath(output_file_path))
# else:
#     print("No 'data' key found in JSON")


Filtered data has been saved to: /Users/margaretguo/Documents/clinical_trial_llm/trial_processing_code/filtered_interventional_trials.json


In [93]:
# len(filtered_data['data']), len(interventional_trials), len(active_trials),len(active_interventional_trials), len(all_data)


(2688, 20894, 6575, 5731, 22898)

In [32]:
# active_trials = active_trials#['data']

In [94]:
# idx = [i for i in range(len(interventional_trials)) if interventional_trials[i]['nct_id']=="NCT05554367"]

# idx

[12]

In [95]:
# i = 12
# trial = interventional_trials[12]
# if 'diseases' in trial:
#     diseases = trial['diseases']
#     diseases_new = process_diseases(diseases)

TRIAL
Invasive Ovarian Micropapillary Serous Carcinoma
False
C105556
TRIAL
Malignant Solid Tumor
True
C132146
TRIAL
Stage IV Pancreatic Cancer
False
C134930
TRIAL
Stage IV Ovarian Cancer
False
C139977
TRIAL
Pancreatic Cancer
False
C3850


In [22]:
# diseases[0].keys()

In [18]:
def process_diseases(diseases):
    structured_dict = {
        'inclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        },
        'exclusion': {
            'TREE': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []},
            'TRIAL': {'nci_thesaurus_concept_id': [], 'ancestors': [],'parents':[], 'synonyms': [], 'names': []}
        }
    }

    for dx in diseases:
        if dx['inclusion_indicator']=='TRIAL':
            print(dx['inclusion_indicator'])
            print(dx['name'])
            print(dx['is_lead_disease'])
            #print(dx['nci_thesaurus_concept_id'])
            
        # Default to 'inclusion' and 'TREE' if keys are missing or values are None
        eligibility = dx.get('is_lead_disease', False)
        indicator = dx.get('inclusion_indicator', 'TREE')

        # Additional check to ensure eligibility and indicator have valid values
        if eligibility not in structured_dict:
            eligibility = 'inclusion'  # Default to 'inclusion' if the value is not recognized
        if indicator not in structured_dict[eligibility]:
            indicator = 'TREE'  # Default to 'TREE' if the value is not recognized
        concept_id = dx['nci_thesaurus_concept_id']
        ancestors = dx.get('ancestors', [])
        parents = dx.get('parents', [])
        synonyms = dx.get('synonyms', [])
        name = dx.get('name', '')
        
        if name is not None:
            entry = structured_dict[eligibility][indicator]
            entry['synonyms'].extend(synonyms)  # Same for synonyms
            if indicator == "TRIAL":
                print(concept_id)
                print("=========================================")
            entry['nci_thesaurus_concept_id'].append(concept_id)
            entry['ancestors'].extend(ancestors)  # Assuming you want to flatten the ancestors list

            entry['names'].append(name)
            entry['parents'].extend(parents)
    # Convert sets back to lists before returning
    for eligibility_status in structured_dict:
        for indicator_type in structured_dict[eligibility_status]:
            for key in structured_dict[eligibility_status][indicator_type]:
                structured_dict[eligibility_status][indicator_type][key] = list(set(structured_dict[eligibility_status][indicator_type][key]))

    return structured_dict

In [19]:
%%time
# Assuming active_trials is your list of trials
s = 0
# interventional_trials
for trial in interventional_trials:
    s+=1
    if 'biomarkers' in trial:
        biomarkers = trial['biomarkers']
        trial['biomarkers_new'] = process_biomarkers(biomarkers)


# Assuming active_trials is your list of trials
s = 0
for trial in interventional_trials:
    s+=1
    if 'prior_therapy' in trial:
        prior_therapies = trial['prior_therapy']
        trial['prior_therapy_new'] = process_prior_therapy(prior_therapies)

s = 0
for trial in interventional_trials:
    s+=1
    if 'diseases' in trial:
        diseases = trial['diseases']
        trial['diseases_new'] = process_diseases(diseases)
        
s = 0
for trial in interventional_trials:
    s+=1
    if 'arms' in trial:
        arms = trial['arms']
        trial['interventions_new'] = process_interventions(arms)        
        

TRIAL
Spindle Cell/Sclerosing Rhabdomyosarcoma
False
C121654
TRIAL
Metastatic Rhabdomyosarcoma
False
C166414
TRIAL
Metastatic Embryonal Rhabdomyosarcoma
False
C183250
TRIAL
Solid Alveolar Rhabdomyosarcoma
False
C27493
TRIAL
Alveolar Rhabdomyosarcoma
False
C3749
TRIAL
Spindle Cell Rhabdomyosarcoma
False
C6519
TRIAL
Embryonal Rhabdomyosarcoma
True
C8971
TRIAL
Botryoid-Type Embryonal Rhabdomyosarcoma
False
C9150
TRIAL
Stage IV Lung Cancer
False
C136482
TRIAL
Recurrent Non-Small Cell Lung Cancer
True
C7783
TRIAL
Stage IV Penile Cancer
False
C140087
TRIAL
Stage IVB Prostate Cancer
False
C140175
TRIAL
Stage IV Renal Cell Cancer
False
C140326
TRIAL
Stage IV Bladder Cancer
False
C140424
TRIAL
Stage IV Urethral Cancer
False
C140463
TRIAL
Metastatic Bladder Carcinoma
False
C156062
TRIAL
Metastatic Bladder Squamous Cell Carcinoma
False
C157750
TRIAL
Metastatic Bladder Plasmacytoid Urothelial Carcinoma
False
C157751
TRIAL
Metastatic Bladder Lipid-Rich Urothelial Carcinoma
False
C157754
TRIAL
Metas

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



CPU times: user 5.15 s, sys: 158 ms, total: 5.31 s
Wall time: 5.29 s


In [20]:
# def find_trials_with_previous_therapies(active_trials, params):
#     key_list = params.keys()
#     if any("previous_treatments" in key for key in key_list):
#         # Extract the codes from the query
#         #first have to do for past meds that are inclusion/exclusion
#         inclusion_codes = []
#         exclusion_codes = []
#         if "include_previous_treatments" in key_list:
#             inclusion_codes = [bm['code'] for bm in params["include_previous_treatments"]]
#         if "exclude_previous_treatments" in key_list:
#             exclusion_codes = [bm['code'] for bm in params["exclude_previous_treatments"]]
#         # Initialize a list to hold trials with overlapping biomarkers
#         matching_trials = []
        
#         # Loop through each trial in active_trials
#         for trial in active_trials:
#             # Check if 'biomarkers_new' exists and has 'inclusion' and 'TRIAL'
#             if 'prior_therapy_new' in trial and \
#             'inclusion' in trial['prior_therapy_new'] and \
#             'TRIAL' in trial['prior_therapy_new']['inclusion']:
            
#                 # Extract the codes for the trial's biomarkers under inclusion and TRIAL
#                 trial_inclusion_codes = trial['prior_therapy_new']['inclusion']['TRIAL']['nci_thesaurus_concept_id']+trial['prior_therapy_new']['inclusion']['TRIAL']['parents']
#                 trial_exclusion_codes = trial['prior_therapy_new']['exclusion']['TRIAL']['nci_thesaurus_concept_id']+trial['prior_therapy_new']['exclusion']['TRIAL']['parents']
#                 # Check for any overlap in codes
#                 if (any(code in inclusion_codes for code in trial_inclusion_codes)) | (any(code in exclusion_codes for code in trial_exclusion_codes)==False):
#                     matching_trials.append(trial)
#     else:
#         matching_trials = []
#     return matching_trials

# def find_trials_with_inteventions(active_trials, params):
#     key_list = params.keys()
#     if any("study_treatments" in key for key in key_list):
#         # Extract the codes from the query
#         #first have to do for past meds that are inclusion/exclusion
#         inclusion_codes = set([])
#         exclusion_codes = set([])
#         if "study_treatments_to_include" in key_list:
#             inclusion_codes = set([bm['code'] for bm in params["study_treatments_to_include"]])
#         if "study_treatments_to_exclude" in key_list:
#             exclusion_codes = set([bm['code'] for bm in params["study_treatments_to_exclude"]])
#         # Initialize a list to hold trials with overlapping biomarkers
#         matching_trials = []
        
#         # Loop through each trial in active_trials
#         for trial in active_trials:
#             # Check if 'biomarkers_new' exists and has 'inclusion' and 'TRIAL'
#             if ('interventions_new' in trial) and \
#             ('inclusion' in trial['interventions_new']) and \
#             ('TRIAL' in trial['interventions_new']['inclusion']):
            
#                 # Extract the codes for the trial's biomarkers under inclusion and TRIAL
#                 trial_codes = set(trial['interventions_new']['inclusion']['TRIAL']['nci_thesaurus_concept_id']+trial['interventions_new']['inclusion']['TRIAL']['descendants'])
#                 tree_codes = set(trial['interventions_new']['inclusion']['TREE']['nci_thesaurus_concept_id']+trial['interventions_new']['inclusion']['TREE']['descendants'])
#                 all_cur_trial_codes = trial_codes.union(tree_codes)
#                 # Check for any overlap in codes
#                 if (len(all_cur_trial_codes.intersection(inclusion_codes))>0) and (len(all_cur_trial_codes.intersection(exclusion_codes))==0):
#                     matching_trials.append(trial)
#     else:
#         matching_trials = []
#     return matching_trials

# def find_trials_with_overlapping_biomarkers(active_trials, params):

#     key_list = params.keys()
#     if any("biomarker" in key for key in key_list):
#         # Extract the codes from the query
#         #first have to do for past meds that are inclusion/exclusion
#         inclusion_codes = []
#         exclusion_codes = []
#         if "include_biomarkers" in key_list:
#             inclusion_codes = [bm['code'] for bm in params["include_biomarkers"]]
#         if "exclude_biomarkers" in key_list:
#             exclusion_codes = [bm['code'] for bm in params["exclude_biomarkers"]]
#         # Initialize a list to hold trials with overlapping biomarkers
#         matching_trials = []
        
#         # Loop through each trial in active_trials
#         for trial in active_trials:
#             # Check if 'biomarkers_new' exists and has 'inclusion' and 'TRIAL'
#             if 'biomarkers_new' in trial and \
#             'inclusion' in trial['biomarkers_new'] and \
#             'TRIAL' in trial['biomarkers_new']['inclusion']:
            
#                 # Extract the codes for the trial's biomarkers under inclusion and TRIAL
#                 trial_codes = trial['biomarkers_new']['inclusion']['TRIAL']['nci_thesaurus_concept_id']+trial['biomarkers_new']['inclusion']['TRIAL']['descendants']
                
#                 # Check for any overlap in codes
#                 if (any(code in inclusion_codes for code in trial_codes)) and (any(code in exclusion_codes for code in trial_codes)==False):
#                     matching_trials.append(trial)
#         if len(matching_trials)<5:
#             for trial in active_trials:
#             # Check if 'biomarkers_new' exists and has 'inclusion' and 'TRIAL'
#                 if 'biomarkers_new' in trial and \
#                 'inclusion' in trial['biomarkers_new'] and \
#                 'TRIAL' in trial['biomarkers_new']['inclusion']:
#                     # Extract the codes for the trial's biomarkers under inclusion and TRIAL
#                     trial_codes = trial['biomarkers_new']['inclusion']['TRIAL']['parents']
#                     # Check for any overlap in codes
#                     if (any(code in inclusion_codes for code in trial_codes)) and (any(code in exclusion_codes for code in trial_codes)==False):
#                         matching_trials.append(trial)
#                         s+=1
#     else:
#         matching_trials = []
#     return matching_trials


# def find_trials_with_overlapping_diseases(active_trials, params):
#     key_list = params.keys()
#     if any("diseases" in key for key in key_list):
#         # Extract the codes from the query
#         inclusion_codes = []
#         exclusion_codes = []
#         if "include_diseases" in key_list:
#             inclusion_codes = [bm['code'] for bm in params["include_diseases"]]
#         if "exclude_diseases" in key_list:
#             exclusion_codes = [bm['code'] for bm in params["exclude_diseases"]]
#         # Initialize a list to hold trials with overlapping biomarkers
#         matching_trials = []
#         s = 0
#         st.write('Number of inclusion codes')
#         st.write(str(len(inclusion_codes)))
#         st.write('Number of exclusion codes')
#         st.write(str(len(exclusion_codes)))
#         # Loop through each trial in active_trials
#         st.write('number of active trials')
#         st.write(len(active_trials))
#         for trial in active_trials:
#             # Check if 'biomarkers_new' exists and has 'inclusion' and 'TRIAL'
#             if 'diseases_new' in trial and \
#             'inclusion' in trial['diseases_new'] and \
#             'TRIAL' in trial['diseases_new']['inclusion']:
            
#                 # Extract the codes for the trial's biomarkers under inclusion and TRIAL
#                 trial_codes = trial['diseases_new']['inclusion']['TRIAL']['nci_thesaurus_concept_id']+trial['diseases_new']['inclusion']['TRIAL']['descendants']
#                 # Check for any overlap in codes
#                 if (any(code in inclusion_codes for code in trial_codes)) and (any(code in exclusion_codes for code in trial_codes)==False):
#                     matching_trials.append(trial)
#                     s+=1
#         #may be overly specific for some diseases, want to try to back out
#         if len(matching_trials)<5:
#             for trial in active_trials:
#             # Check if 'biomarkers_new' exists and has 'inclusion' and 'TRIAL'
#                 if 'diseases_new' in trial and \
#                 'inclusion' in trial['diseases_new'] and \
#                 'TRIAL' in trial['diseases_new']['inclusion']:
#                     # Extract the codes for the trial's biomarkers under inclusion and TRIAL
#                     trial_codes = trial['diseases_new']['inclusion']['TRIAL']['parents']
#                     # Check for any overlap in codes
#                     if (any(code in inclusion_codes for code in trial_codes)) and (any(code in exclusion_codes for code in trial_codes)==False):
#                         matching_trials.append(trial)
#                         s+=1

#     else:
#         matching_trials = []
    
#     return matching_trials

In [21]:
# disease_trials = find_trials_with_overlapping_diseases(interventional_trials, params)
# intervention_trials = find_trials_with_inteventions(interventional_trials, params)


In [51]:
# codes = [item['code'] for item in params['study_treatments_to_include']]

In [52]:
# thesaurus = pd.read_csv('nci-streamlit-app\Thesaurus.txt',sep='\t',header=None)
# thesaurus.columns = ['Serial','url','code','Name','Description','no1','no2','type','Synonyms']

In [53]:

# thesaurus = thesaurus[['Serial','url','code','Name','type']]
# keep_types = ['Cell or Molecular Dysfunction','Neoplastic Process','Finding','Therapeutic or Preventive Procedure','Chemical Viewed Functionally']
# thesaurus = thesaurus[thesaurus['type'].isin(keep_types)].reset_index(drop=True)

In [54]:
# thesaurus = st.session_state.thesaurus
# keep_types = {
#     "disease": ['Neoplastic Process'],
#     "biomarker": ['Cell or Molecular Dysfunction', 'Finding'],
#     "intervention": ['Therapeutic or Preventive Procedure', 'Chemical Viewed Functionally','Pharmacologic Substance',
#                     'Inorganic Chemical|Pharmacologic Substance','Organic Chemical|Pharmacologic Substance','Pharmacologic Substance']
# }

# relevant_types = [keep_types[k] for k in keep_types if k in concept_type][0]
# thesaurus = thesaurus[thesaurus['type'].isin(relevant_types)]
# valid_codes = set(thesaurus['Serial'])

In [118]:
# import requests
# url = 'https://api-evsrest.nci.nih.gov/api/v1/concept/ncit/search'
# params = {
#     'term': 'PARP Inhibitor',
#     'pageSize': 30,
#     "type":"fuzzy",
#     #"include":"minimal",
#     "terminology":"ncit",
#     "definitionSource":"NCI",
#     #"synonymType":"FULL_SYN"
# }
# response = requests.get(url, params=params)

In [23]:
save_data_to_file({"total": len(interventional_trials), 
                   "data": interventional_trials}, 
                  f'interventional_trials_simple_dx_bm_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.json')

get some info on what the keys of interventional trials are

In [25]:
keys_arr = sorted(interventional_trials[0].keys())
keys_to_values = {}
for key in keys_arr:
    print('******')
    print(key)
    key_list = []
    for trial in interventional_trials:
        key_list.append(trial[key])
    try:
        key_list_counter = Counter(key_list)
        keys_to_values[key] = key_list_counter
        if len(key_list_counter)<20:
            print(key_list_counter)
    except:
        print('ERROR: not typable list')

******
_current_trial_status_sort_order
Counter({9: 6371, 0: 5588, 7: 4446, 8: 3021, 6: 836, 10: 639, 4: 327, 2: 136, 3: 136, 1: 49, 5: 5})
******
_phase_sort_order
Counter({2: 6620, 4: 5297, 7: 4119, 3: 2727, 0: 1870, 5: 494, 1: 216, 6: 211})
******
_primary_purpose_sort_order
Counter({0: 15897, 1: 2304, 4: 1185, 3: 1014, 6: 447, 2: 278, 5: 185, 7: 179, None: 65})
******
_study_protocol_type_sort_order
Counter({0: 21554})
******
acronym
******
active_sites_count
******
amendment_date
******
anatomic_sites
ERROR: not typable list
******
arms
ERROR: not typable list
******
associated_studies
ERROR: not typable list
******
biomarkers
ERROR: not typable list
******
biomarkers_new
ERROR: not typable list
******
brief_summary
******
brief_title
******
ccr_id
******
central_contact
ERROR: not typable list
******
classification_code
Counter({None: 17008, 'Efficacy': 2724, 'SAFETY': 939, 'Safety/Efficacy': 777, 'Pharmacodynamics': 38, 'Pharmacokinetics': 25, 'NA': 21, 'Bio-availability': 17, '

can we quantify how many of the biomarker trials are stopped due to low acrual

technical
 - 'keys_to_values['why_study_stopped']' --> clustering 2334
 - 

In [116]:
keys_to_values['why_study_stopped']

Counter({None: 17196,
         'Not provided': 129,
         'low accrual': 66,
         'PI Request': 62,
         'Slow accrual': 51,
         'Sponsor decision': 48,
         'slow accrual': 40,
         'Low accrual': 36,
         'Administratively Complete': 36,
         'PI request': 26,
         'PI decision': 26,
         'Inadequate accrual rate': 26,
         'Business decision': 23,
         'Low Accrual': 22,
         'Lack of accrual': 20,
         'Sponsor Decision': 20,
         'Business reasons': 19,
         'poor accrual': 16,
         '< 75% accrual': 16,
         'Strategic considerations': 15,
         'Unknown': 15,
         'Slow Accrual': 14,
         '<75% accrual': 13,
         'Funding': 13,
         '0 participant accrual': 13,
         'Drug supply issues': 12,
         'Lack of efficacy': 11,
         'Business Decision': 11,
         'Business Reasons': 11,
         'Lack of enrollment': 10,
         'PI left institution': 10,
         'Funding unavailab

dict_keys(['other_ids', 'amendment_date', 'keywords', 'dcp_id', 'interventional_model', 'lead_org', 'eligibility', 'sites', 'completion_date_type_code', 'detail_description', 'official_title', '_phase_sort_order', 'collaborators', 'associated_studies', 'outcome_measures', 'phase', 'central_contact', 'primary_purpose', 'number_of_arms', '_study_protocol_type_sort_order', 'nct_id', 'biomarkers', 'classification_code', 'current_trial_status_date', 'diseases', '_primary_purpose_sort_order', 'protocol_id', 'active_sites_count', 'lead_org_cancer_center', 'arms', 'study_model_code', 'nci_id', 'why_study_stopped', 'brief_summary', 'brief_title', 'status_history', 'study_population_description', 'sampling_method_code', 'minimum_target_accrual_number', 'prior_therapy', '_current_trial_status_sort_order', 'start_date', 'record_verification_date', 'ctep_id', 'current_trial_status', 'study_model_other_text', 'masking', 'acronym', 'nci_programs', 'nci_funded', 'anatomic_sites', 'ccr_id', 'start_date_type_code', 'principal_investigator', 'study_source', 'completion_date', 'study_subtype_code', 'study_protocol_type', 'biomarkers_new', 'prior_therapy_new', 'diseases_new', 'interventions_new'])


# things we care about

******
_current_trial_status_sort_order
Counter({0: 6575})
******
_phase_sort_order
Counter({7: 2021, 2: 1700, 4: 1373, 3: 797, 0: 423, 5: 139, 1: 66, 6: 56})
******
_primary_purpose_sort_order
Counter({0: 4158, 7: 728, 1: 640, 4: 361, 3: 261, 6: 183, 2: 131, 5: 93, None: 20})
******
_study_protocol_type_sort_order
Counter({0: 5731, 1: 844})
******
acronym
******
active_sites_count
******
amendment_date
******
anatomic_sites
ERROR: not typable list
******
arms
ERROR: not typable list
******
associated_studies
ERROR: not typable list
******
biomarkers
ERROR: not typable list
******
biomarkers_new
ERROR: not typable list
******
brief_summary
******
brief_title
******
ccr_id
******
central_contact
ERROR: not typable list
******
classification_code
Counter({None: 6358, 'Efficacy': 142, 'Safety/Efficacy': 37, 'SAFETY': 30, 'NA': 3, 'Pharmacodynamics': 2, 'Pharmacokinetics': 2, 'Pharmacokinetics/dynamics': 1})
******
collaborators
ERROR: not typable list
******
completion_date
******
completion_date_type_code
Counter({'ANTICIPATED': 4840, None: 1734, 'ACTUAL': 1})
******
ctep_id
******
current_trial_status
Counter({'Active': 6575})
******
current_trial_status_date
******
dcp_id
******
detail_description
******
diseases
ERROR: not typable list
******
diseases_new
ERROR: not typable list
******
eligibility
ERROR: not typable list
******
interventional_model
Counter({'Single Group': 2617, 'Parallel': 2163, 'SEQUENTIAL_ASSIGNMENT': 933, None: 785, 'Cross-over': 47, 'Factorial': 30})
******
interventions_new
ERROR: not typable list
******
keywords
ERROR: not typable list
******
lead_org
******
lead_org_cancer_center
******
masking
ERROR: not typable list
******
minimum_target_accrual_number
******
nci_funded
Counter({'Indirect': 5437, 'Direct': 1138})
******
nci_id
******
nci_programs
ERROR: not typable list
******
nct_id
******
number_of_arms
******
official_title
******
other_ids
ERROR: not typable list
******
outcome_measures
ERROR: not typable list
******
phase
Counter({'NA': 2021, 'II': 1700, 'I': 1373, 'I_II': 797, 'III': 423, 'O': 139, 'II_III': 66, 'IV': 56})
******
primary_purpose
Counter({'TREATMENT': 4158, 'OTHER': 728, 'SUPPORTIVE_CARE': 640, 'DIAGNOSTIC': 361, 'PREVENTION': 261, 'HEALTH_SERVICES_RESEARCH': 183, 'SCREENING': 131, 'BASIC_SCIENCE': 93, 'DEVICE': 20})
******
principal_investigator
******
prior_therapy
ERROR: not typable list
******
prior_therapy_new
ERROR: not typable list
******
protocol_id
******
record_verification_date
******
sampling_method_code
Counter({None: 5644, 'NON_PROBABILITY_SAMPLE': 872, 'PROBABILITY_SAMPLE': 59})
******
sites
ERROR: not typable list
******
start_date
******
start_date_type_code
Counter({'ACTUAL': 6485, 'ANTICIPATED': 90})
******
status_history
ERROR: not typable list
******
study_model_code
Counter({None: 5601, 'COHORT': 675, 'CASE_ONLY': 149, 'CASE_CONTROL': 83, 'OTHER': 41, 'FAMILY_BASED': 13, 'ECOLOGIC_OR_COMMUNITY_STUDIES': 11, 'CASE_CROSSOVER': 2})
******
study_model_other_text
Counter({None: 6538, 'Need to be reviewed by scientific abstractor': 12, 'Not currently abstracted by DCP': 9, 'Needs to be reviewed by scientific abstractor': 3, 'Controlled': 1, 'Only for Epidemiological; cancer control/behavioral; observational; ancillary; correlative; or other biological studies': 1, 'Cross-sectional': 1, 'Randomized controlled trial': 1, 'Parallel': 1, 'Cross-sectional followed by longitudinal cohort': 1, 'Staggered start implementation (mixed methods study)': 1, 'Peripheral blood sample with DNA and RNA; Tumor Tissue (of any histology); Adrenocortical tumors;': 1, 'Observational': 1, 'Registry': 1, 'Delphi Study': 1, 'Survey': 1, 'registry': 1})
******
study_population_description
******
study_protocol_type
Counter({'Interventional': 5731, 'Non-interventional': 844})
******
study_source
Counter({'Institutional': 3445, 'Industrial': 1934, 'Externally Peer Reviewed': 913, 'National': 283})
******
study_subtype_code
Counter({None: 5601, 'OBSERVATIONAL': 899, 'ANCILLARY_CORRELATIVE': 75})
******
why_study_stopped
Counter({None: 6575})

In [71]:
trial['outcome_measures']

[{'timeframe': 'Up to 4 hours',
  'name': 'Observed plasma concentrations',
  'description': 'Will be assessed by Pearson correlation and quantification of the predictive performance. The values from plasma will be considered the observed concentration.',
  'type_code': 'PRIMARY'},
 {'timeframe': 'Up to 4 hours',
  'name': 'Predictive performance of the home-based sampling',
  'description': 'Will be assessed by Pearson correlation and quantification of the predictive performance. The predictive performance of the home-based sampling methods will be assessed by calculating the median percentage prediction error (MPPE).',
  'type_code': 'PRIMARY'}]

# 1. get possible diseases and their ids

only did once in July 2024 - updated Apr 2025

In [6]:
# Define the base URL of the API
BASE_URL = "https://clinicaltrialsapi.cancer.gov/api/v2/"
# endpoint = "trials"
# If needed, add your API key and other headers here
HEADERS = {
    "Content-Type": "application/json",
    "x-api-key":  "NibhGQRcr84CrgDHkego51aqrNioOR1F7k6f7dNp"  # Uncomment and replace YOUR_API_KEY with your actual API key if needed
}
PARAMS= {}

In [7]:
# get the possible diseases
endpoint='diseases'
addon_filter = '?type=maintype'#&type_not=subtype'
url = f"{BASE_URL}{endpoint}{addon_filter}"
response = requests.get(url, headers=HEADERS, params=PARAMS)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response data
    diseases = response.json()
    print("Success:", len(diseases['data']))
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}, Response: {response.text}")

Success: 125


In [8]:
diseases_df = pd.DataFrame.from_dict(diseases['data'])
diseases_df.shape

(125, 7)

In [9]:
# diseases_df.name.values

In [24]:
diseases_df.to_csv('diseases_df.csv')

## manual review of ontology

there are 125 maintypes + subtypes and 51 maintypes only


- go through the csv and start grouping ids that we want together #i.e. gallbladder cancer and cholangiocarcinoma go together
- noting that maintype is often sufficient
- making a dictionary mapping <str> to <list> of <str>
    - key: name of disease group
    - value: list of ncid_ids, always take ancestor id first 

In [29]:
diseases_groups = pd.read_excel('diseases_df_new.xlsx', sheet_name=1, header=0).fillna('')
diseases_groups['code_list'] = diseases_groups.apply(lambda row: [ x for x in [row['code1'],row['code2'],row['code3'] ]if x !=''],axis=1)
display(diseases_groups)
group_to_ids = pd.Series(diseases_groups.code_list.values, index=diseases_groups.group_name.values).to_dict()

,group_name,name1,code1,name2,code2,name3,code3,code_list
0,biliary,Ampulla of Vater Cancer,C3908,Gallbladder Cancer,C3844,Bile Duct Cancer (including Cholangiocarcinoma),C27814,"[C3908, C3844, C27814]"
1,anal,Anal Cancer,C9291,,,,,[C9291]
2,appendix,Appendix Cancer,C9330,,,,,[C9330]
3,bladder_uu,Bladder Cancer,C4912,Ureter Carcinoma,C8993,Urethral Cancer,C9106,"[C4912, C8993, C9106]"
4,bone,Bone Cancer,C4016,,,,,[C4016]
5,breast,Breast Cancer,C4872,Paget Disease,C7073,,,"[C4872, C7073]"
6,cervix_vv,Cervical Cancer,C9039,Vaginal Cancer,C3917,Vulvar Cancer,C4866,"[C9039, C3917, C4866]"
7,colon,Colorectal (Colon or Rectal) Cancer,C2955,,,,,[C2955]
8,embryo,Embryonal Neoplasm,C3264,Trophoblastic Tumor,C3422,Germ Cell Tumor,C3708,"[C3264, C3422, C3708]"
9,esophageal,Esophageal Cancer,C3513,,,,,[C3513]


# 1. get possible biomarkers
and their ids

 
 For example, if you request `/biomarkers?maintype=C4872` this will give you all biomarkers that are in trials where Breast Cancer(C4872) is among the diseases in each trial. Note that maintype is a trials endpoint parameter related to diseases.nci_thesaurus_concept_id.

In [26]:
# reference files

hgnc_df = pd.read_table('hgnc_non_alt_loci_set.txt')
nci_hgnc_df = pd.read_table('NCIt-HGNC_Mapping.txt',names=['nci_code','hgnc_id'])
print(nci_hgnc_df.shape)

hgnc_df_brief = hgnc_df[['hgnc_id','symbol']].drop_duplicates()
# hgnc_df_brief.shape, hgnc_df.shape
nci_hgnc_df = nci_hgnc_df.merge(hgnc_df_brief, how='left',on='hgnc_id').fillna('')
print(nci_hgnc_df[nci_hgnc_df.hgnc_id!=''].shape)
display(nci_hgnc_df[:5])

#mapping functions
hgnc_to_symbol = pd.Series(hgnc_df.symbol.values, index=hgnc_df.hgnc_id.values).to_dict()
nci_to_symbol = pd.Series(nci_hgnc_df.symbol.values, index=nci_hgnc_df.nci_code.values).to_dict()


(6004, 2)
(6004, 3)


/var/folders/v0/7508blv5757906b15gssdjzm0000gn/T/ipykernel_5579/2376929057.py:3: DtypeWarning: Columns (32,34,38,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  hgnc_df = pd.read_table('hgnc_non_alt_loci_set.txt')


,nci_code,hgnc_id,symbol
0,C101088,HGNC:18037,ARID2
1,C101091,HGNC:20893,BCOR
2,C101095,HGNC:24224,CDK12
3,C101098,HGNC:23438,NUTM2A
4,C101101,HGNC:23445,NUTM2B


In [27]:
# Define the base URL of the API
BASE_URL = "https://clinicaltrialsapi.cancer.gov/api/v2/"
# endpoint = "trials"
# If needed, add your API key and other headers here
HEADERS = {
    "Content-Type": "application/json",
    "x-api-key":  "NibhGQRcr84CrgDHkego51aqrNioOR1F7k6f7dNp"  # Uncomment and replace YOUR_API_KEY with your actual API key if needed
}
PARAMS= {}

In [32]:
%%time
# get the possible biomarkers for each maintype disease
disease_to_biomarkers = {}
for idx, row in diseases_groups.iterrows():
    print(row['group_name'],row['code_list'])
    biomarkers_list = []
    for code in row['code_list']:
        
        endpoint='biomarkers/?maintype='
        url = f"{BASE_URL}{endpoint}{code}"
        response = requests.get(url, headers=HEADERS, params=PARAMS)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response data
            biomarkers = response.json()
            biomarkers_list+=biomarkers['data']
    
            print("Success:", len(biomarkers['data']))
            time.sleep(2)
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}, Response: {response.text}")
            
    # raise #testing
    
    #looping through all disease biomarkers first make them unique
    print('total biomarkers #:', len(biomarkers_list))
    biomarkers_dict = {}
    for idx, biomarker in enumerate(biomarkers_list):
        if len(biomarker['codes'])==1:
            biomarkers_dict[biomarker['codes'][0]] = biomarker
    print('unique biomarkers #:', len(biomarkers_dict))
    
    #mapping form nci code to gene symbol if the nci code has an hgnc gene symbol associated with the code 
    biomarker_names = {} 
    for code, biomarker in biomarkers_dict.items():
        if code  in nci_to_symbol:
            if nci_to_symbol[code]=='':
                print(code, biomarker['name'])
            biomarker_names[code] = nci_to_symbol[code]
    print('# gene symbols:', len(biomarker_names)  )
    
    disease_to_biomarkers[row['group_name']] = biomarker_names
    # raise

biliary ['C3908', 'C3844', 'C27814']
Success: 79
Success: 156
Success: 371
total biomarkers #: 606
unique biomarkers #: 375
# gene symbols: 63
anal ['C9291']
Success: 127
total biomarkers #: 127
unique biomarkers #: 127
# gene symbols: 16
appendix ['C9330']
Success: 69
total biomarkers #: 69
unique biomarkers #: 69
# gene symbols: 9
bladder_uu ['C4912', 'C8993', 'C9106']
Success: 638
Success: 213
Success: 194
total biomarkers #: 1045
unique biomarkers #: 649
# gene symbols: 149
bone ['C4016']
Success: 457
total biomarkers #: 457
unique biomarkers #: 457
# gene symbols: 88
breast ['C4872', 'C7073']
Success: 1135
Success: 0
total biomarkers #: 1135
unique biomarkers #: 1135
C71259 HLA-DRB3 Gene
# gene symbols: 213
cervix_vv ['C9039', 'C3917', 'C4866']
Success: 331
Success: 75
Success: 87
total biomarkers #: 493
unique biomarkers #: 331
# gene symbols: 52
colon ['C2955']
Success: 910
total biomarkers #: 910
unique biomarkers #: 910
# gene symbols: 154
embryo ['C3264', 'C3422', 'C3708']
Su

In [34]:
nci_to_diseases = defaultdict(list)
for disease, nci_to_sym in disease_to_biomarkers.items():
    for nci in nci_to_sym.keys():
        nci_to_diseases[nci].append(disease)

nci_biomarker_df = {}
for nci, diseases in nci_to_diseases.items():
    diseases_str = '|'.join(diseases)
    nci_biomarker_df[nci] = {'nci_id':nci, 'sym':nci_to_symbol[nci],'num_dz':len(diseases), 'diseases_str':diseases_str}
nci_biomarker_df = pd.DataFrame.from_dict(nci_biomarker_df,orient='index')
nci_biomarker_df

,nci_id,sym,num_dz,diseases_str
C17756,C17756,ERBB2,31,biliary|anal|appendix|bladder_uu|bone|breast|c...
C38349,C38349,ESR1,31,biliary|anal|appendix|bladder_uu|bone|breast|c...
C17060,C17060,HRAS,30,biliary|anal|appendix|bladder_uu|bone|breast|c...
C25785,C25785,KRAS,30,biliary|anal|appendix|bladder_uu|bone|breast|c...
C18345,C18345,MDM2,15,biliary|bladder_uu|breast|embryo|eye|malig_hem...
...,...,...,...,...
C20564,C20564,TINF2,1,otherdisease
C114613,C114613,UNC13D,1,otherdisease
C43236,C43236,WAS,1,otherdisease
C102959,C102959,WRAP53,1,otherdisease


In [38]:

pkl_file = f'disease_to_biomarkers_{datetime.now().strftime("%Y-%m-%d")}.pkl'
print(pkl_file)
with open(pkl_file, 'wb') as file:
    pickle.dump(disease_to_biomarkers, file)

disease_to_biomarkers_2024-12-26.pkl


In [39]:
biomarker_filename = f'nci_biomarker_{datetime.now().strftime("%Y-%m-%d")}_df.csv'
print(biomarker_filename)
nci_biomarker_df.to_csv(biomarker_filename)

nci_biomarker_2024-12-26_df.csv
